<a href="https://colab.research.google.com/github/HasibAlMuzdadid/Machine-Learning-and-Deep-Learning-Projects/blob/main/fraud%20detection%20%5Bgradient%20checking%5D/fraud_detection_using_gradient_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Authenticity using Gradient Checking**

You are part of a team working to make mobile payments available globally, and are asked to build a deep learning model to detect fraud--whenever someone makes a payment, you want to see if the payment might be fraudulent, such as if the user's account has been taken over by a hacker.


We already know that backpropagation is quite challenging to implement, and sometimes has bugs. Because this is a mission-critical application, your company's CEO wants to be really certain that your implementation of backpropagation is correct. Your CEO says, *Give me proof that your backpropagation is actually working!*. To give this reassurance, you are going to use **gradient checking**.


In [ ]:
import numpy as np

In [ ]:
# helper function
def sigmoid(x):
  s = 1/(1+np.exp(-x))
  return s

def relu(x):
  s = np.maximum(0,x)  
  return s

In [ ]:
def dictionary_to_vector(parameters):

  keys = []
  count = 0
  for key in ["W1", "b1", "W2", "b2", "W3", "b3"]:
        
    # flatten parameter
    new_vector = np.reshape(parameters[key], (-1,1))
    keys = keys + [key]*new_vector.shape[0]
        
    if count == 0:
      theta = new_vector
    else:
      theta = np.concatenate((theta, new_vector), axis=0)
    
    count = count + 1

  return theta, keys


def vector_to_dictionary(theta):

  parameters = {}
  parameters["W1"] = theta[:20].reshape((5,4))
  parameters["b1"] = theta[20:25].reshape((5,1))
  parameters["W2"] = theta[25:40].reshape((3,5))
  parameters["b2"] = theta[40:43].reshape((3,1))
  parameters["W3"] = theta[43:46].reshape((1,3))
  parameters["b3"] = theta[46:47].reshape((1,1))

  return parameters


def gradients_to_vector(gradients):
 
  count = 0
  for key in ["dW1", "db1", "dW2", "db2", "dW3", "db3"]:
    # flatten parameter
    new_vector = np.reshape(gradients[key], (-1,1))
        
    if count == 0:
      theta = new_vector
    else:
      theta = np.concatenate((theta, new_vector), axis=0)
    
    count = count + 1

  return theta

def gradient_check_n_test_case(): 
 
  x = np.random.randn(4,3)
  y = np.array([1, 1, 0])
  W1 = np.random.randn(5,4) 
  b1 = np.random.randn(5,1) 
  W2 = np.random.randn(3,5) 
  b2 = np.random.randn(3,1) 
  W3 = np.random.randn(1,3) 
  b3 = np.random.randn(1,1) 
  parameters = {"W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2,
                "W3": W3,
                "b3": b3}

    
  return x, y, parameters  

**One Dimensional Gradient Checking**

In [ ]:
# forward propagation

def forward_propagation(x, theta):

  J = theta * x
  
    
  return J

In [ ]:
# backward propagation

def backward_propagation(x, theta):
  
  dtheta = x
    

  return dtheta

In [ ]:
# gradient check

def gradient_check(x, theta, epsilon=1e-7, print_msg=False):
   
  # Compute gradapprox
  thetaplus = theta + epsilon                               # Step 1
  thetaminus = theta - epsilon                              # Step 2
  J_plus = forward_propagation(x, thetaplus)                # Step 3
  J_minus = forward_propagation(x, thetaminus)              # Step 4
  gradapprox = (J_plus - J_minus) / (2*epsilon)             # Step 5
    
    
  # Check if gradapprox is close enough to the output of backward_propagation()
  grad = backward_propagation(x, theta)
    
  numerator = np.linalg.norm(grad-gradapprox)                                  # Step 1'
  denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)              # Step 2'
  difference = numerator / denominator                                         # Step 3'
    
  if print_msg:
    if difference > 2e-7:
      print ("\033[91m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
      print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")
    
  return difference

In [ ]:
x, theta = 2, 4
difference = gradient_check(x, theta, print_msg=True)

Your backward propagation works perfectly fine! difference = 2.919335883291695e-10


**N-Dimensional Gradient Checking**

In [ ]:
def forward_propagation_n(X, Y, parameters):
   
  # retrieve parameters
  m = X.shape[1]
  W1 = parameters["W1"]
  b1 = parameters["b1"]
  W2 = parameters["W2"]
  b2 = parameters["b2"]
  W3 = parameters["W3"]
  b3 = parameters["b3"]

  # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
  Z1 = np.dot(W1, X) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = relu(Z2)
  Z3 = np.dot(W3, A2) + b3
  A3 = sigmoid(Z3)

  # Cost
  log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
  cost = 1. / m * np.sum(log_probs)
    
  cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
  return cost, cache

In [ ]:
def backward_propagation_n(X, Y, cache):

  m = X.shape[1]
  (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
  dZ3 = A3 - Y
  dW3 = 1. / m * np.dot(dZ3, A2.T)
  db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
  dA2 = np.dot(W3.T, dZ3)
  dZ2 = np.multiply(dA2, np.int64(A2 > 0))
  dW2 = 1. / m * np.dot(dZ2, A1.T) * 2                          # error 1
  db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
  dA1 = np.dot(W2.T, dZ2)
  dZ1 = np.multiply(dA1, np.int64(A1 > 0))
  dW1 = 1. / m * np.dot(dZ1, X.T)
  db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True)             # error 2
    
  gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
               "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
               "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
  return gradients

In [ ]:
# GRADED FUNCTION: gradient_check_n

def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7, print_msg=False):
    
  # Set-up variables
  parameters_values, _ = dictionary_to_vector(parameters)
    
  grad = gradients_to_vector(gradients)
  num_parameters = parameters_values.shape[0]
  J_plus = np.zeros((num_parameters, 1))
  J_minus = np.zeros((num_parameters, 1))
  gradapprox = np.zeros((num_parameters, 1))
    
  # Compute gradapprox
  for i in range(num_parameters):
        
    # Compute J_plus[i]. Inputs: "parameters_values, epsilon". Output = "J_plus[i]".
    # "_" is used because the function you have to outputs two parameters but we only care about the first one

    thetaplus = parameters_values.copy()                                                   # Step 1
    thetaplus[i][0] = thetaplus[i][0] + epsilon                                            # Step 2
    J_plus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))            # Step 3
        
        
    # Compute J_minus[i]. Inputs: "parameters_values, epsilon". Output = "J_minus[i]".
    thetaminus = parameters_values.copy()                                              # Step 1
    thetaminus[i][0] = thetaminus[i][0] - epsilon                                      # Step 2        
    J_minus[i], _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))      # Step 3
        
    # Compute gradapprox[i]
    gradapprox[i] = (J_plus[i] - J_minus[i]) / (2*epsilon)
    
  # Compare gradapprox to backward propagation gradients by computing difference.
  numerator = np.linalg.norm(grad-gradapprox)                                         # Step 1
  denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                     # Step 2
  difference = numerator / denominator                                                # Step 3
    
  if print_msg:
    if difference > 2e-7:
      print ("\033[91m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
    else:
      print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

  return difference

In [ ]:
X, Y, parameters = gradient_check_n_test_case()

cost, cache = forward_propagation_n(X, Y, parameters)
gradients = backward_propagation_n(X, Y, cache)
difference = gradient_check_n(parameters, gradients, X, Y, 1e-7, True)
expected_values = [0.2850931567761623, 1.1890913024229996e-07]

There is a mistake in the backward propagation! difference = 8.615988854416132e-05
